<a href="https://colab.research.google.com/github/rafavidal1709/projeto-aplicado-iii/blob/main/BERTimbau.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from transformers import BertTokenizer, BertModel
import pandas as pd

# Carregar o tokenizador e o modelo pré-treinado BERTimbau Large
tokenizer = BertTokenizer.from_pretrained('neuralmind/bert-large-portuguese-cased')
model = BertModel.from_pretrained('neuralmind/bert-large-portuguese-cased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [2]:
# Exemplo de tokenização de uma frase
sentence = "A degradação ambiental e o abandono de espaços comunitários limitam o desenvolvimento social e econômico das comunidades. Este projeto tem como objetivo revitalizar a antiga Escolinha do João Cota, transformando-a no primeiro Centro Agroecológico e Cultural de São Tomé das Letras. O centro promoverá educação ambiental, cultura agroecológica e o fortalecimento da agricultura familiar orgânica, envolvendo diretamente a comunidade local."
inputs = tokenizer(sentence, return_tensors="pt")

# Obter as saídas do modelo
outputs = model(**inputs)

# As saídas contêm os embeddings gerados pelo BERTimbau
display(outputs.last_hidden_state.shape)  # Todos os tokens em um tensor de (1, n, 1024) dimensões
display(outputs.pooler_output.shape)      # Token especial CLS para representação do texto completo, utilizado para clustering e classificação.
print("\n")
display(outputs.pooler_output)

torch.Size([1, 81, 1024])

torch.Size([1, 1024])

tensor([[-0.0485, -0.9990,  0.5399,  ..., -0.0164,  0.8627,  0.0265]],
       grad_fn=<TanhBackward0>)

In [3]:
import json

# Caminho do arquivo
file_path = '/content/realistic_denuncias_ambientais_5000_v2.json'

# Leitura do arquivo JSON
with open(file_path, 'r') as file:
    data = json.load(file)

# Exibindo um exemplo da variável 'data' para verificar a estrutura
data[:2]  # Exibir os dois primeiros itens para conferir

[{'text': 'Queimaram tudo aqui na fazenda do Sítio do João pra aumentar o espaço pro gado. Já aconteceu ano passado.',
  'tags': ['incendio_criminoso', 'APP', 'impacto ambiental', 'urgente']},
 {'text': 'Queimada ilegal em uma fazenda próxima a Norte, está destruindo a flora local e ameaçando animais selvagens.',
  'tags': ['queimadas', 'APP', 'impacto ambiental', 'já ocorreu']}]

In [4]:
import torch

# Verificar se o GPU está disponível
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Mover o modelo para o GPU
model.to(device)

for i in range(len(data)):
    # Tokenizar o texto e mover as entradas para o GPU
    inputs = tokenizer(data[i]['text'], return_tensors="pt").to(device)

    # Fazer a inferência no GPU
    outputs = model(**inputs)

    # Mover os resultados de volta para a CPU para manipulação
    #data[i]['tokens'] = outputs.last_hidden_state[0].cpu().tolist()
    data[i]['CLS_token'] = outputs.pooler_output[0].cpu().tolist()

    if (i+1) % 100 == 0:
      print(f'{i+1} / {len(data)}')


100 / 5000
200 / 5000
300 / 5000
400 / 5000
500 / 5000
600 / 5000
700 / 5000
800 / 5000
900 / 5000
1000 / 5000
1100 / 5000
1200 / 5000
1300 / 5000
1400 / 5000
1500 / 5000
1600 / 5000
1700 / 5000
1800 / 5000
1900 / 5000
2000 / 5000
2100 / 5000
2200 / 5000
2300 / 5000
2400 / 5000
2500 / 5000
2600 / 5000
2700 / 5000
2800 / 5000
2900 / 5000
3000 / 5000
3100 / 5000
3200 / 5000
3300 / 5000
3400 / 5000
3500 / 5000
3600 / 5000
3700 / 5000
3800 / 5000
3900 / 5000
4000 / 5000
4100 / 5000
4200 / 5000
4300 / 5000
4400 / 5000
4500 / 5000
4600 / 5000
4700 / 5000
4800 / 5000
4900 / 5000
5000 / 5000


In [5]:
# Salvando a variável 'data' como um arquivo JSON
output_file_path = '/content/realistic_denuncias_ambientais_5000_v2_tokenized.json'

with open(output_file_path, 'w') as outfile:
    json.dump(data, outfile)

# Forçando o download do arquivo no Colab
from google.colab import files
files.download(output_file_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [10]:
from sklearn.neighbors import NearestNeighbors
import numpy as np

# Criando uma matriz com todos os vetores de saída (CLS_token)
CLS_matrix = np.array([item['CLS_token'] for item in data])

# Definindo o número de vizinhos (k) e criando o modelo KNN
k = 30  # Número de vizinhos mais próximos que você deseja encontrar
knn = NearestNeighbors(n_neighbors=k, metric='euclidean')

# Ajustando o modelo com a matriz de vetores (1024 dimensões)
knn.fit(CLS_matrix)

NearestNeighbors(metric='euclidean', n_neighbors=30)

In [37]:
# Supondo que 'target_vector' é o vetor de consulta que você deseja encontrar os vizinhos mais próximos
#target_vector = CLS_matrix[1024]  # Exemplo de vetor de consulta com 1024 dimensões

text = input("Escreva uma denúncia:\n")
print("\n--------------------------------------\n")

# Tokenizar o texto e mover as entradas para o GPU
inputs = tokenizer(text, return_tensors="pt").to(device)

# Fazer a inferência no GPU
outputs = model(**inputs)

# Mover os resultados de volta para a CPU para manipulação
target_vector = np.array(outputs.pooler_output[0].cpu().tolist())

# Encontrando os 'k' vizinhos mais próximos do 'target_vector'
distances, indices = knn.kneighbors([target_vector])

# Mostrando os vizinhos mais próximos
for i in range(len(indices[0])):
    j = indices[0][i]
    print(f"{j}: {', '.join(data[j]['tags'])}\n{data[j]['text']}\nDistância euclidiana: {distances[0][i]}\n")

Escreva uma denúncia:
Meu vizinho Jorge jogou glifosato na nascente

--------------------------------------

315: veneno_agricultura, APP, impacto ambiental, já ocorreu
Tá todo mundo ficando doente aqui com esse veneno que jogam nas plantações perto da vila.
Distância euclidiana: 5.413188552564574

340: veneno_agricultura, APP, impacto ambiental, urgente
Tá todo mundo ficando doente aqui com esse veneno que jogam nas plantações perto da vila.
Distância euclidiana: 5.413188552564574

403: veneno_agricultura, APP, impacto ambiental, já ocorreu
Tá todo mundo ficando doente aqui com esse veneno que jogam nas plantações perto da vila.
Distância euclidiana: 5.413188552564574

341: veneno_agricultura, APP, impacto ambiental, já ocorreu
Tá todo mundo ficando doente aqui com esse veneno que jogam nas plantações perto da vila.
Distância euclidiana: 5.413188552564574

866: veneno_agricultura, APP, impacto ambiental, urgente
Tá todo mundo ficando doente aqui com esse veneno que jogam nas plantaçõe